# Definir funciones e imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({
    'text.usetex': True,            # Usar LaTeX para todo el texto
    'font.size': 14,                # Tamaño de la fuente general
    'axes.titlesize': 14,           # Tamaño de los títulos de los ejes
    'axes.labelsize': 14,           # Tamaño de las etiquetas de los ejes
    'xtick.labelsize': 14,          # Tamaño de las etiquetas de los ticks del eje x
    'ytick.labelsize': 14,          # Tamaño de las etiquetas de los ticks del eje y
    'legend.fontsize': 14,          # Tamaño de la fuente en las leyendas
    'figure.titlesize': 16,         # Tamaño del título principal de la figura
    'text.latex.preamble': r'\usepackage{amsmath}'  # Permitir usar amsmath en LaTeX
})


Function to dissect the dictionary filename for parameters

In [ ]:

# Función para extraer parámetros desde dic_name
def vars_from_dictname(fname):
    # Convertir a string
    cadena = str(fname)
    # Encontrar posiciones
    inicio = cadena.find('S_')
    fin = cadena.rfind('.mat')
    # Extraer el substring
    fname = cadena[inicio:fin]

    # Dividir el nombre del archivo por los guiones bajos
    partes = fname.split('_')

    # Inicializar un diccionario para almacenar los valores
    sk_params = {}

    # Extraer las sk_params
    for i in range(len(partes)):
        if partes[i] == 'm':
            sk_params['M'] = int(partes[i + 1])
        elif partes[i] == 'n':
            sk_params['N'] = int(partes[i + 1])
        elif partes[i] == 'vsc':
            sk_params['n_v_scales'] = int(partes[i + 1])
        elif partes[i] == 'hsc':
            sk_params['n_h_scales'] = int(partes[i + 1])
        elif partes[i] == 'bases':
            bases = [float(partes[i + 1]), float(partes[i + 2])]
            sk_params['bases'] = bases
        elif partes[i] == 'thV':
            sk_params['n_v_thetas'] = int(partes[i + 1])
        elif partes[i] == 'thH':
            sk_params['n_h_thetas'] = int(partes[i + 1])

    return sk_params


Function to load all perform files in a folder and arrange them in a pandas dataframe

In [ ]:

# Función para cargar los archivos .mat y extraer los datos
def load_perform_files_from_folder(main_dir):
    # Inicializar lista para almacenar el contenido de los archivos .mat
    list_perform = []

    # Recorrer todos los subdirectorios y archivos
    for root, dirs, files in os.walk(main_dir):
        for file in files:
            # Si el archivo es 'perform.mat', lo cargamos
            if file == 'perform.mat':
                # Construir la ruta completa al archivo
                file_path = os.path.join(root, file)
                # Cargar el archivo .mat (squeeze_me solves problems with strings and added dimensions in np.arrays and scalars)
                mat_data = sio.loadmat(file_path, squeeze_me=True)

                # Aplicar squeeze a todos los elementos de mat_data
                mat_data_squeezed = {key: value for key, value in mat_data.items() if not key.startswith('__')}

                # Extraer parámetros desde dic_name
                dic_name = mat_data_squeezed['dic_name']  # Usamos squeeze
                sk_params = vars_from_dictname(dic_name)

                # Añadir parámetros extraídos del nombre del archivo a mat_data
                combined_data = {**mat_data_squeezed, **sk_params}

                # Añadir el contenido del archivo combinado a la lista
                list_perform.append(combined_data)

    # Convertir la lista en un DataFrame de pandas
    df = pd.DataFrame(list_perform)

    print(f"Se han cargado {len(list_perform)} archivos 'perform.mat'.")
    return df

# # Ruta principal donde se encuentran los subdirectorios
# main_dir = "outputs/"
# # Cargar los archivos y construir el DataFrame
# df_perform = load_perform_files_from_folder(main_dir)
# # Mostrar las primeras filas del DataFrame
# print(df_perform.head())

# Cargar archivos

In [ ]:
# df_ss_mics020 = load_perform_files_from_folder(r"outputs/rm_0.2/shearlets")
# df_bs_mics020 = load_perform_files_from_folder(r"outputs/rm_0.2/tan")

# df_ss_mics033 = load_perform_files_from_folder(r"outputs/rm_0.33/shearlets")
# df_bs_mics033 = load_perform_files_from_folder(r"outputs/rm_0.33/tan")



# Check padding in shearlets 
Image (M0, N0)=(100, 100) 

In [ ]:
# Crear la tabla pivote
# tabla_pivote = df_ss_mics033.pivot(index='M', columns='N', values='NMSE')
# tabla_pivote

# tabla_pivote = df_ss_mics020.pivot(index='M', columns='N', values='NMSE')
# tabla_pivote


# Compare ShearletSystem and BS in similar conditions
+ Padding 128x128
+ 4 scales in H & V cones
    - $\alpha = 1/\sqrt{2}$
+ Applied to same image 
    - Balder $T_{start}=1000$ and M0=100
    - jittering mask, ratio_mics=0.2


Plot of NMSE.  
I have chosen $\beta^*$ in the x axis because the shearlets do not have another parameter as boostlet systems to order the data,  
and $\beta^*$ is a unique value for each NMSE.

In [ ]:
Tstart=0
ratio_mics=0.2
nmics=int(ratio_mics*100.)
str_title = rf'Balder $n_{{mics}}={nmics}$, $T_{{start}}={Tstart}$, $(M,N)=(128, 128)$'

df = load_perform_files_from_folder(rf"outputs/T{Tstart}/rm_{ratio_mics}/tan_like_ss/T_{Tstart}_M0_100")
df_ss = load_perform_files_from_folder(rf"outputs/T{Tstart}/rm_{ratio_mics}/shearlets")


ss_df = df_ss[ (df_ss['M']==128) & (df_ss['N']==128) ]
bs_df_a = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==3)]
bs_df_b = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==7)]
bs_df_c = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==15)]

xax = 'beta_star'
yax = 'NMSE'

colors = {
    'shearlets': '#1f77b4',    # Azul para ShearletsSyst
    'bs_df_a': '#ff7f0e',      # Naranja para la primera variación de BS
    'bs_df_b': '#2ca02c',      # Verde para la segunda variación de BS
    'bs_df_c': '#d62728'       # Rojo oscuro para la tercera variación de BS
}

fig, ax = plt.subplots()
ax.plot(ss_df[xax], ss_df[yax], 'v', label='ShearletSyst', color=colors['shearlets'])
ax.plot(bs_df_a[xax], bs_df_a[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_a["n_h_thetas"].iloc[0]}$', color=colors['bs_df_a'])
ax.plot(bs_df_b[xax], bs_df_b[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_b["n_h_thetas"].iloc[0]}$', color=colors['bs_df_b'])
ax.plot(bs_df_c[xax], bs_df_c[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_c["n_h_thetas"].iloc[0]}$', color=colors['bs_df_c'])

# Graficar NMSE_lin
ax.plot(ss_df[xax], ss_df['NMSE_lin'], '*', alpha=0.7, label=r'NMSE$_{lin}$')
ax.plot(bs_df_a[xax], bs_df_a['NMSE_lin'], '*', alpha=0.7)
ax.plot(bs_df_b[xax], bs_df_b['NMSE_lin'], '*', alpha=0.7)
ax.plot(bs_df_c[xax], bs_df_c['NMSE_lin'], '*', alpha=0.7)


# Añadir texto con el valor de 'n_v_thetas' cerca de cada punto en bs_df_a
for i, txt in enumerate(bs_df_a['n_v_thetas']):
    ax.text(bs_df_a[xax].iloc[i], bs_df_a[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_a'], ha='left', va='bottom')
    
for i, txt in enumerate(bs_df_b['n_v_thetas']):
    ax.text(bs_df_b[xax].iloc[i], bs_df_b[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_b'], ha='left', va='bottom')

for i, txt in enumerate(bs_df_c['n_v_thetas']):
    ax.text(bs_df_c[xax].iloc[i], bs_df_c[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_c'], ha='left', va='bottom')

# Etiquetas de los ejes
ax.set_xlabel(r'$\beta^*$')   # Etiqueta con formato LaTeX
ax.set_ylabel('NMSE')
ax.grid(True)

# Mostrar leyenda y gráfica
# plt.legend(loc='lower left', bbox_to_anchor=(0.65, 0.1), fontsize='small', framealpha=0.8)  # Fuera de la gráfica
plt.legend(loc='best')  # Fuera de la gráfica

plt.suptitle(str_title)
plt.tight_layout(rect=(0, 0, 0.9, 1)) # solves a problem where the right part of the plot is missing in .eps
plt.savefig(f'./figs_paper/NMSE_beta_star_ss_bstan_BalderT{Tstart}_m{nmics}_M128.eps', format='eps', bbox_inches='tight')
plt.show()



##  Check dicts

In [ ]:
from mod_RIRIS_func import load_sk
from mod_plotting_utilities import plot_ls_images

folder_dict = r'ss_saved_dicts/'
file_dict = r'SS_m_128_n_128.mat'
temp = load_sk(folder=folder_dict, file=file_dict, build_dict=None)
indices = [0,1,3,4,6,7,8,10,11,12,13,15,16,17,21,22,26,27,33,34,40] # 1, 3, 4, 6, 8, 10 , 11, 12, 17
sc_ss = sum(temp[:, :, i]**2 for i in indices)

# folder_dict = r'saved_dicts/tan_dicts/sc_like_shearlets/'
folder_dict = r'saved_dicts/tan_dicts/M128_bases/'
# file_dict = r'BS_m_128_n_128_vsc_3_hsc_3_bases_0.7071_0.7071_thV_3_thH_3.mat'
file_dict = r'BS_m_128_n_128_vsc_0_hsc_0_bases_0.1_0.1_thV_3_thH_3.mat'
temp = load_sk(folder=folder_dict, file=file_dict, build_dict=None)
sc_bs_3_3 = temp[:,:,0]

# folder_dict = r'saved_dicts/tan_dicts/sc_like_shearlets/'
# file_dict = r'BS_m_128_n_128_vsc_3_hsc_3_bases_0.7071_0.7071_thV_3_thH_7.mat'
file_dict = r'BS_m_128_n_128_vsc_2_hsc_2_bases_0.5_0.5_thV_3_thH_7.mat'
temp = load_sk(folder=folder_dict, file=file_dict, build_dict=None)
sc_bs_3_7 = temp[:,:,0]

# folder_dict = r'saved_dicts/tan_dicts/sc_like_shearlets/'
# file_dict = r'BS_m_128_n_128_vsc_3_hsc_3_bases_0.7071_0.7071_thV_3_thH_15.mat'
file_dict = r'BS_m_128_n_128_vsc_19_hsc_19_bases_0.9_0.9_thV_3_thH_15.mat'
temp = load_sk(folder=folder_dict, file=file_dict, build_dict=None)
sc_bs_3_15 = temp[:,:,0]

images = [sc_ss, sc_bs_3_3, sc_bs_3_7, sc_bs_3_15]
titles = ["ShearletSystem", r"BS $n_{\theta_H}$ = 3", r"BS $n_{\theta_H}$ = 7", r"BS $n_{\theta_H}$ = 15"]
fig, ax = plt.subplots(1,4, figsize=(16,6))
for i, im in enumerate(images):
    ax[i].imshow(np.abs(im))
    ax[i].axis('off')
    ax[i].set_title(titles[i])
    ax[i].plot(np.arange(128), np.arange(128), 'k')
    ax[i].plot(np.arange(128), np.arange(128, 0, -1), 'k')
plt.savefig('./figs_paper/elements_aligned_with_diag.eps', format='eps', bbox_inches='tight')
plt.show()


# Compare scatter of the results

In [ ]:

# str_title = rf'Balder $n_{{mics}}={20}$, $T_{{start}}={1000}$, $(M,N)=(128, 128)$'

# df_ss_mics020 = load_perform_files_from_folder(r"outputs/rm_0.2/shearlets")
# df = load_perform_files_from_folder(r"outputs/rm_0.2/tan_like_ss")
# # df = load_perform_files_from_folder(r"outputs/rm_0.2/tan_like_ss/T_1000_M0_100")

# ss_df = df_ss_mics020[ (df_ss_mics020['M']==128) & (df_ss_mics020['N']==128) ]
# bs_df_a = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==3)]
# bs_df_b = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==7)]
# bs_df_c = df[(df['M']==128) & (df['N']==128) & (df['n_v_scales']==3) & (df['n_h_scales']==3) & (df['n_h_thetas']==15)]

# xax = 'beta_star'
# yax = 'NMSE'

# colors = {
#     'shearlets': '#1f77b4',    # Azul para ShearletsSyst
#     'bs_df_a': '#ff7f0e',      # Naranja para la primera variación de BS
#     'bs_df_b': '#2ca02c',      # Verde para la segunda variación de BS
#     'bs_df_c': '#d62728'       # Rojo oscuro para la tercera variación de BS
# }

# fig, ax = plt.subplots(figsize=(16,6))
# ax.plot(ss_df[xax], ss_df[yax], 'v', label='ShearletSyst', color=colors['shearlets'])
# ax.plot(bs_df_a[xax], bs_df_a[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_a["n_h_thetas"].iloc[0]}$', color=colors['bs_df_a'])
# ax.plot(bs_df_b[xax], bs_df_b[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_b["n_h_thetas"].iloc[0]}$', color=colors['bs_df_b'])
# ax.plot(bs_df_c[xax], bs_df_c[yax], 'o', label=fr'BS $n_{{\theta_H}} = {bs_df_c["n_h_thetas"].iloc[0]}$', color=colors['bs_df_c'])

# # Graficar NMSE_lin
# ax.plot(ss_df[xax], ss_df['NMSE_lin'], '*', alpha=0.7)
# ax.plot(bs_df_a[xax], bs_df_a['NMSE_lin'], '*', alpha=0.7)
# ax.plot(bs_df_b[xax], bs_df_b['NMSE_lin'], '*', alpha=0.7)
# ax.plot(bs_df_c[xax], bs_df_c['NMSE_lin'], '*', alpha=0.7)


# # Añadir texto con el valor de 'n_v_thetas' cerca de cada punto en bs_df_a
# # for i, txt in enumerate(bs_df_a['n_v_thetas']):
# #     ax.text(bs_df_a[xax].iloc[i], bs_df_a[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_a'], ha='left', va='bottom')
    
# # for i, txt in enumerate(bs_df_b['n_v_thetas']):
# #     ax.text(bs_df_b[xax].iloc[i], bs_df_b[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_b'], ha='left', va='bottom')

# # for i, txt in enumerate(bs_df_c['n_v_thetas']):
# #     ax.text(bs_df_c[xax].iloc[i], bs_df_c[yax].iloc[i], str(txt), fontsize=10, color=colors['bs_df_c'], ha='left', va='bottom')

# # Etiquetas de los ejes
# ax.set_xlabel(r'$\beta^*$')   # Etiqueta con formato LaTeX
# ax.set_ylabel('NMSE')
# ax.grid(True)

# # Mostrar leyenda y gráfica
# plt.legend(loc='lower left', bbox_to_anchor=(1.01, 0.1), fontsize='small', framealpha=0.8)  # Fuera de la gráfica
# # plt.legend(loc='best')  # Fuera de la gráfica

# plt.suptitle(str_title)
# plt.tight_layout(rect=(0, 0, 0.9, 1)) # solves a problem where the right part of the plot is missing in .eps
# # plt.savefig('./figs_paper/NMSE_beta_star_ss_bstan_BalderT1000_m20_M128.eps', format='eps', bbox_inches='tight')
# plt.show()

In [ ]:
bs_df_aa = bs_df_a[bs_df_a['n_v_thetas']==3]
bs_df_ab = bs_df_a[bs_df_a['n_v_thetas']==7]
bs_df_ac = bs_df_a[bs_df_a['n_v_thetas']==15]

bs_df_ba = bs_df_b[bs_df_b['n_v_thetas']==3]
bs_df_bb = bs_df_b[bs_df_b['n_v_thetas']==7]
bs_df_bc = bs_df_b[bs_df_b['n_v_thetas']==15]

bs_df_ca = bs_df_c[bs_df_c['n_v_thetas']==3]
bs_df_cb = bs_df_c[bs_df_c['n_v_thetas']==7]
bs_df_cc = bs_df_c[bs_df_c['n_v_thetas']==15]

print(f"mean NMSE shearlets:  {ss_df['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(3, 3):  {bs_df_aa['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(3, 7):  {bs_df_ab['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(3, 15):  {bs_df_ac['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(7, 3):  {bs_df_ba['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(7, 7):  {bs_df_bb['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(7, 15):  {bs_df_bc['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(15, 3):  {bs_df_ca['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(15, 7):  {bs_df_cb['NMSE'].mean()}")
print(f"mean NMSE BS (h, v)=(15, 15):  {bs_df_cc['NMSE'].mean()}")



In [ ]:

print(f"std NMSE shearlets:  {ss_df['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(3, 3):  {bs_df_aa['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(3, 7):  {bs_df_ab['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(3, 15):  {bs_df_ac['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(7, 3):  {bs_df_ba['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(7, 7):  {bs_df_bb['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(7, 15):  {bs_df_bc['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(15, 3):  {bs_df_ca['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(15, 7):  {bs_df_cb['NMSE'].std()}")
print(f"std NMSE BS (h, v)=(15, 15):  {bs_df_cc['NMSE'].std()}")


In [ ]:
plt.figure(figsize=(10, 6))

# Graficar el campo NMSE
plt.plot(df['NMSE'], 'o', color='b', label='NMSE')

# Agregar etiquetas con el campo 'bases' en cada punto
for i, base in enumerate(df['dic_name']):
    plt.text(i, df['NMSE'].iloc[i], str(base), fontsize=9, ha='right')

# Configurar etiquetas y título del gráfico
plt.xlabel("Índice")
plt.ylabel("NMSE")
plt.title("Gráfico de NMSE con etiquetas de 'bases'")
plt.legend()

# Mostrar el gráfico
plt.show()

Estadisticas:

In [ ]:
Tstart=1000
ratio_mics=0.2
nmics=int(ratio_mics*100.)

df_ss = load_perform_files_from_folder(rf"outputs/stats_runs/shearlets/rm_{ratio_mics}/T_{Tstart}_M0_100")
df = load_perform_files_from_folder(rf"outputs/stats_runs/tan_like_ss/rm_{ratio_mics}/T_{Tstart}_M0_100")



In [ ]:
df = load_perform_files_from_folder(rf"outputs/BigImage")
df

# Tests con distintos r

In [ ]:
Tstart=1000
ratio_mics=0.33

nmics=int(ratio_mics*100)
str_title = rf'Balder $n_{{mics}}={nmics}$, $T_{{start}}={Tstart}$, $(M,N)=(128, 128)$'

df = load_perform_files_from_folder(rf"outputs/T{Tstart}/rm_{ratio_mics}/tan/")



Mirar todos los outputs, marcados por el $n_{\theta_H}$

In [ ]:

df_nh15 = df[ df['n_h_thetas']==15 ]
df_nh7 = df[ df['n_h_thetas']==7 ]
df_nh3 = df[ df['n_h_thetas']==3 ]

fig, ax = plt.subplots()
for dfi in [df_nh15, df_nh7, df_nh3]:
    ax.plot(dfi['NMSE'], 'o', label=rf"$n_{{\theta_H}} ={dfi['n_h_thetas'].iloc[0]}$")
ax.legend()
ax.set_xlabel("index")
ax.set_ylabel("NMSE")
plt.show()


Los mejores resultados son para $n_{\theta_H}=15$, extraigo esos datos y sigo filtrando.

In [ ]:
df15 = df[ (df['n_h_thetas']==15) ]
unique_values = df15['n_v_thetas'].unique()

# Crear un gráfico para cada valor de 'n_v_thetas'
plt.figure(figsize=(6, 4))

for n_v_theta in unique_values:
    subset = df15[df15['n_v_thetas'] == n_v_theta]    
    # Graficar los datos del subconjunto
    plt.plot(subset['NMSE'], marker='o', label=fr'$n_{{\theta_V}}={n_v_theta}$')

# plt.title(f'Gráfico para n_v_thetas = {n_v_theta}')
plt.xlabel('index')
plt.ylabel('NMSE')
plt.legend()
plt.grid(True)

# Mostrar o guardar el gráfico
plt.show()



Ahora cojo el mejor caso que es $n_{\theta_V}=15$ y estudio para distintas bases

In [ ]:
# Filtrar por `n_v_thetas == 15` y hacer una copia explícita
df1515 = df15[df15['n_v_thetas'] == 15].copy()

# Convertir listas en la columna `bases` a tuplas y obtener valores únicos
df1515['bases_tuple'] = df1515['bases'].apply(lambda x: tuple(x) if isinstance(x, list) else x)
bases_to_filter = df1515['bases_tuple'].dropna().unique()

# Crear un diccionario para almacenar los subconjuntos
filtered_dfs = {
    str(base): df1515[df1515['bases_tuple'] == base]
    for base in bases_to_filter
}

# Graficar los subconjuntos
fig, ax = plt.subplots()

for base, dfi in filtered_dfs.items():
    if not dfi.empty:  # Asegurarse de que el subconjunto no esté vacío
        ax.plot(dfi['NMSE'], 'o', label=rf"$(\alpha_V, \alpha_H)={base}$")

# Configurar leyendas y mostrar
ax.set_title(r"Boostlets with $(n_{\theta_V}, n_{\theta_H})=(15,15)$")
# ax.set_xlabel("index")
ax.set_xticks([])
ax.set_ylabel("NMSE")
ax.legend()
plt.show()

Mejores resultados:
+ $n_{\theta_H} = 15$
+ $n_{\theta_V} = 15$
+ $\alpha_{V} = 0.1$ mejor que $\alpha_{V} = 0.5$ mejor que $\alpha_{V} = 0.9$
    - 0.1 Son escalas muy grandes, solo cabe una isc=0
    - 0.5 escalas medias, caben 3 escalas isc=0,1,2
    - 0.9 escales pequeñas, caben 20 escalas, isc=0,...,19



In [ ]:
print(df1515[['NMSE', 'bases']])